In [16]:
import boto3, operator
import pprint as pp
#from datetime import timedelta
rmclient = boto3.client('robomaker')

In [3]:
response = rmclient.list_simulation_jobs(
    maxResults=100,
    filters=[
        {   
            'name': 'simulationApplicationName',
            'values': [
                'deepracer-simapp-1c7297c6-1289-4ea2-bdf2-f91a0e7bdec1',
            ],
        }
    ]
)
for job in response['simulationJobSummaries']:
    # Get & populate job summary
    job['summary'] = rmclient.describe_simulation_job(job=job['arn'])
                                                                                                   

In [20]:
for job in response['simulationJobSummaries']:
    # Loop split up to reduce calls to AWS while developing script
    
    job['name'] = job['arn'].partition("/")[-1]
    if 'lastStartedAt' in job['summary'].keys():
        job['startTime'] = job['summary']['lastStartedAt']
    else:
        job['startTime'] = None
    job['stopTime'] = None
    if 'lastUpdatedAt' in job['summary'].keys() and job['status'] != "Running":
        job['stopTime'] = job['summary']['lastUpdatedAt']
    if job['startTime'] and job['stopTime']:
        job['duration'] = (job['startTime'] - job['stopTime']).total_seconds()
    else:
        job['duration'] = None
    job['maxduration'] = job['summary']['maxJobDurationInSeconds']
    job['track'] = job['summary']['simulationApplications'][0]['launchConfig']['environmentVariables']['WORLD_NAME']
    
    job['desc'] = "{} - Track: {} - Duration: {}".format(job['name'],job['track'],job['maxduration']/60)
    pp.pprint(job)
    


#pp.pprint(list(map(operator.itemgetter('desc','arn'),response['simulationJobSummaries'])))

{'arn': 'arn:aws:robomaker:us-east-1:244366466142:simulation-job/sim-flwbnpp6kf5s',
 'desc': 'sim-flwbnpp6kf5s - Track: reinvent_base - Duration: 120.0',
 'duration': 98.0,
 'lastUpdatedAt': datetime.datetime(2019, 10, 6, 22, 38, 3, tzinfo=tzlocal()),
 'maxduration': 7200,
 'name': 'sim-flwbnpp6kf5s',
 'robotApplicationNames': [None],
 'simulationApplicationNames': ['deepracer-simapp-1c7297c6-1289-4ea2-bdf2-f91a0e7bdec1'],
 'startTime': datetime.datetime(2019, 10, 6, 22, 39, 41, tzinfo=tzlocal()),
 'status': 'Completed',
 'stopTime': datetime.datetime(2019, 10, 6, 22, 38, 3, tzinfo=tzlocal()),
 'summary': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                                  'content-length': '2652',
                                                  'content-type': 'application/json',
                                                  'date': 'Tue, 08 Oct 2019 '
                                                          '11:34:09 GMT',
       

                                                                                   'SAGEMAKER_SHARED_S3_BUCKET': 'aws-deepracer-3d5f818e-0254-41c9-864f-55465f029f90',
                                                                                   'SAGEMAKER_SHARED_S3_PREFIX': 'DeepRacer-SageMaker-RoboMaker-comm-244366466142-20191004020108-5954a008-bd77-4f03-84a1-09f79e1933a3',
                                                                                   'TARGET_REWARD_SCORE': 'None',
                                                                                   'TRAINING_JOB_ARN': 'arn:aws:deepracer:us-east-1:244366466142:training_job/CUawNQW-TACo5N8U8-ZB5Q',
                                                                                   'WORLD_NAME': 'reinvent_base'},
                                                          'launchFile': 'distributed_training.launch',
                                                          'packageName': 'deepracer_simulation_environ

In [109]:


import ipywidgets as widgets
dropdownlist=list(map(operator.itemgetter('desc','name'),response['simulationJobSummaries']))
simSelectWidget = widgets.Dropdown(
    options=dropdownlist,
    value=dropdownlist[0][1],
    description='RoboMaker Job:',
    disabled=False,
)
display(simSelectWidget)

from IPython.display import display
button = widgets.Button(description="Load Summary")
output = widgets.Output(layout={'border': '1px solid black'})

display(button, output)
#selectedJob=None
def on_button_clicked(b):
    selectedJob = list(filter(lambda job: job['name'] == simSelectWidget.value, response['simulationJobSummaries']))[0]
    with output:
        output.clear_output()
        print("SimID: {}\nTrack: {}".format(selectedJob['name'],selectedJob['track']))

button.on_click(on_button_clicked)


Dropdown(description='RoboMaker Job:', options=(('sim-flwbnpp6kf5s - Track: reinvent_base - Duration: 120.0', …

Button(description='Load Summary', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [112]:
# Scratchpad

##pp.pprint(list(filter(lambda job: job['name'] == simSelectWidget.value, response['simulationJobSummaries'])))
#selectedJob = list(filter(lambda job: job['name'] == simSelectWidget.value, response['simulationJobSummaries']))[0]
#pp.pprint(selectedJob)

In [113]:
# Fetching hyper parameters
log_group = "/aws/robomaker/SimulationJobs"

client = boto3.client('logs')
kwargs = {
        'logGroupName': log_group,
        'logStreamNamePrefix': selectedJob['name'],
        'limit': 10000,
    }
kwargs['startTime'] = 1451490400000
kwargs['endTime'] = 2000000000000
resp = client.filter_log_events(**kwargs)

In [120]:
import json
parsingHyperParams = False
hyperParamsRaw=""
for event in resp['events']:
    if parsingHyperParams:
        hyperParamsRaw += event['message']
    if parsingHyperParams and event['message'] == "}":
        parsingHyperParams = False
    if not parsingHyperParams and event['message'] == 'Using the following hyper-parameters':
        parsingHyperParams = True


    
        
    

In [125]:
pp.pprint(json.loads("{}"))

{}
